In [1]:
# 重新建立模型
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers,optimizers,datasets,Sequential


class SIFTnet(keras.Model):
    def __init__(self, Num_dense, dropout_value, Num_classes):
        super(SIFTnet, self).__init__()
        
        self.layer1 = Sequential([layers.experimental.preprocessing.Rescaling(1. / 255),])
            
        self.layer2 = Sequential([layers.Conv2D(64, (3, 3), activation='relu',padding = 'same'),])
        self.layer3 = Sequential([layers.MaxPooling2D(2, 2),])

        self.layer4 = Sequential([layers.Conv2D(128, (3, 3), activation='relu',padding = 'same'),])
        self.layer5 = Sequential([layers.MaxPooling2D(2, 2),])

        self.layer6 = Sequential([layers.Conv2D(256, (3, 3), activation='relu',padding = 'same'),])
        self.layer7 = Sequential([layers.MaxPooling2D(2, 2),])

        self.layer8 = Sequential([layers.Conv2D(512, (3, 3), activation='relu',padding = 'same'),])
        self.layer9 = Sequential([layers.MaxPooling2D(2, 2),])

        self.layer10 = Sequential([layers.Flatten(),])

        self.layer11 = Sequential([layers.Dense(Num_dense, activation='relu'),])
        self.layer12 = Sequential([layers.Dropout(dropout_value)])
        self.layer13 = Sequential([layers.Dense(Num_classes, activation = 'softmax')])
        

        
        
    def call(self, inputs, training = None):
        x = self.layer1(inputs)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x)
        x = self.layer10(x)
        
        x = self.layer11(x)        
        x = self.layer12(x)
        x = self.layer13(x)
        
        return x
        
        
# def main():
#     model = SIFTnet(32,0.1,4)
#     model.build(input_shape=(None, 256, 256, 3))
#     model.summary()
    
    
# if __name__ == '__main__':
#     main()

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from time import *
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Activation,Dropout,Flatten,Dense
from tensorflow.keras.optimizers import SGD


# 数据集加载函数，指明数据集的位置并统一处理为imgheight*imgwidth的大小，同时设置batch
def data_load(data_dir, test_data_dir, img_height, img_width, batch_size):
    # 加载训练集
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        label_mode='categorical',
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)
    # 加载测试集
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        test_data_dir,
        label_mode='categorical',
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)
    class_names = train_ds.class_names
    # 返回处理之后的训练集、验证集和类名
    return train_ds, val_ds, class_names


# 构建CNN模型
def model_load(IMG_SHAPE=(256, 256, 3), class_num=4):
    # 搭建模型

    
# model
    model = SIFTnet(128,0.1,class_num)
    model.build(input_shape=(None, 256, 256, 3))


    
    # 输出模型信息
    model.summary()
    # 指明模型的训练参数，优化器为sgd优化器，损失函数为交叉熵损失函数
    initial_learning_rate = 0.005
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=5401,
        decay_rate=0.9,
        staircase=True)

    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr_schedule,momentum=0.9),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
#     model.compile(optimizer=SGD(lr=1e-3,momentum=0.9),loss='categorical_crossentropy',metrics=['accuracy'])
    # 返回模型
    return model


# 展示训练过程的曲线
def show_loss_acc(history, pic_path,his_path):
    # 从history中提取模型训练集和验证集准确率信息和误差信息
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    with open (his_path,'w')as file:
        for epochs in range(len(acc)):
            file.write(str(round(acc[epochs],6)))
            file.write('  ')
            file.write(str(round(loss[epochs],6)))
            file.write('  ')
            file.write(str(round(val_acc[epochs],6)))
            file.write('  ')
            file.write(str(round(val_loss[epochs],6)))
            file.write('\n')
    
    # 按照上下结构将图画输出
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()), 1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.savefig(pic_path, dpi=100)


def train(epochs, modelsave_name):
    # 开始训练，记录开始时间
    print(modelsave_name)
    begin_time = time()
    # todo 加载数据集， 修改为你的数据集的路径
    train_ds, val_ds, class_names = data_load("Z:\\Jupyter_work\\tensorflow\\Fifthplus\\all_data\\train",
                                            "Z:\\Jupyter_work\\tensorflow\\Fifthplus\\all_data\\val", 256, 256, 32)  # 文件位置
    print(class_names)
    # 加载模型
    model = model_load(class_num=len(class_names))
    # 指明训练的轮数epoch，开始训练
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
    # todo 保存模型， 修改为你要保存的模型的名称
    model.save("modelsave\\"+modelsave_name+'.h5')
    # 记录结束时间
    end_time = time()
    run_time = end_time - begin_time
    print('该循环程序运行时间：', run_time, "s")  
    # 绘制模型训练过程图
    show_loss_acc(history, "result\\"+modelsave_name+'_loss.png', 
                     'history\\'+modelsave_name+'_history.txt')


if __name__ == '__main__':
    print('训练集3,学习率0.005_0.9,丢失率0.1，epochs=20')
    train(epochs=20,modelsave_name ='SIFT-net3-FC32-0.1-0.005_0.9')


训练集3,学习率0.005_0.9,丢失率0.1，epochs=20
SIFT-net3-FC32-0.1-0.005_0.9
Found 172804 files belonging to 4 classes.
Found 19196 files belonging to 4 classes.
['class1', 'class2', 'class3', 'class4']
Model: "sif_tnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 256, 256, 64)      1792      
_________________________________________________________________
sequential_2 (Sequential)    (None, 128, 128, 64)      0         
_________________________________________________________________
sequential_3 (Sequential)    (None, 128, 128, 128)     73856     
_________________________________________________________________
sequential_4 (Sequential)    (None, 64, 64, 128)       0         
_________________________________________________________________


KeyboardInterrupt: 